# **CSE 234: Machine Learning Systems - PA 1**
Welcome to the first PA of CSE234 - Winter 2025!

This assignment solely requires Python programming; no C++ coding is involved. Make sure to refer to the final section of this notebook for details on how to submit your assignment.

Avoid posting your completed work on any public platforms (such as GitHub). This assignment must be completed individually and is not intended for group work.

No GPU is needed for this assignment. You can use this notebook on CoLab to develop.

### **Testing and Grading**

We have provided a suite of public test scripts located in the tests/ directory that you can use to verify your implementation. Additionally, your assignment will be evaluated using a series of private tests. The marks you receive for each section will depend on how many of these private tests your code passes. You are allowed to submit your assignment multiple times, but only the last submission will be considered for grading after the deadline. This implies that your scores will not be immediately visible post-submission. We suggest writing your own test scripts to ensure your code functions correctly.

## **Overview**

Automatic differentiation forms the core technique for training machine learning models. In this assignment, you are required to develop a basic automatic differentiation system from scratch. Several functions will be given to you, and you will focus on creating standard operations used in transformers and other ML architectures - namely LayerNorm, ReLU, Softmax, among others.

Let's first go through a run-down of how autodiff works.

The automatic differentiation algorithm in this assignment operates using a computational graph. A computational graph visually represents the sequence of operations needed to compute an expression. You can reference lecture 2, slide 37 for a quick example on how this graph works.

Let's begin by exploring the fundamental concepts and data structures used in the framework. A computational graph is composed of nodes, each representing a distinct computation step in the evaluation of the entire expression.

Each node consists of three components, as shown in auto_diff.py line 6:

*   an operation (field op), specifying the type of computation the node performs.
*   a list of input nodes (field inputs), detailing the sources of input for the computation.
*   optionally, additional "attributes" (field attrs), which vary depending on the node's operation.

These attributes will be discussed in more detail later in this section.

Input nodes in a computational graph can be defined using ad.Variable. For instance, the input variable nodes $x_1$ and $x_2$ might be set up as follows:

```python
import auto_diff as ad

x1 = ad.Variable(name="x1")
x2 = ad.Variable(name="x2")
```

In auto_diff.py (line 81), the ad.Variable class is used to create a node with the operation placeholder and a specified name. Input nodes have empty inputs and attrs:

```python
class Variable(Node):
    def __init__(self, name: str) -> None:
        super().__init__(inputs=[], op=placeholder, name=name)
```

Here, the placeholder operation signifies that the input variable node does not perform any computation. Apart from placeholder, there are other operations defined in auto_diff.py, like add and matmul. You should not create your own instances of these ops.

Returning to our example where
$y = x_1 * x_2 + x_1$, with x1 and x2 already established as input variables, the rest of the graph can be defined using just one line of Python:

```python
y = x1 * x2 + x1
```

This code first creates a node with the operation mul, taking x1 and x2 as its inputs. It then constructs another node with add, which utilizes the result of the multiplication node along with x1 as inputs. Consequently, this computational graph ultimately comprises four nodes.

Important Note

It's important to note that a computational graph (e.g., the four nodes we defined) does not inherently store the actual values of its nodes. The structure of this assignment aligns with the TensorFlow v1 approach that was covered in our lectures. This method contrasts with frameworks like PyTorch, where input tensor values are specified upfront, and the values of intermediate tensors are computed immediately as they are defined.

In our framework, to calculate the value of the output y given the inputs x1 and x2, we utilize the Evaluator class found in auto_diff.py at line 373.

#### **Evaluator**

Here's a walkthrough of how Evaluator works. The constructor of Evaluator accepts a list of nodes that it needs to evaluate. By initiating an Evaluator with:

```evaluator = ad.Evaluator(eval_nodes=[y])```

you are essentially setting up an Evaluator instance designed to compute the value of y. To calculate this, input tensor values are provided via the Evaluator.run method, which you will implement. These input tensors are assumed to be of type numpy.ndarray throughout this assignment. Here’s how it works:

```python
import numpy as np

x1_value = np.array(2)
x2_value = np.array(3)
y_value = evaluator.run(input_dict={x1: x1_value, x2: x2_value})

```

In this process, the run method takes the input values using a dictionary of the form `Dict[Node, numpy.ndarray]`, calculates the value of the node y internally, and outputs the result. For instance, with the input values 2 * 3 + 2 = 8, the expected result for y_value would be `np.ndarray(8)`.

The `Evaluator.run` method is responsible for the forward computation of nodes. Building on what was discussed in the lectures, to calculate the gradient of the output with respect to each input node within a computational graph, we enhance the forward graph with an additional backward component. By integrating both forward and backward graphs, and providing values for the input nodes, the Evaluator can compute the output value, the loss value, and the gradient values for each input node in a single execution of `Evaluator.run`.

You are tasked with implementing the function ```gradients(output_node: Node, nodes: List[Node]) -> List[Node]``` for some of the operators found in auto_diff.py. This function constructs the backward graph needed for gradient computation. It accepts an output node—typically the node representing the loss function in machine learning applications, where the gradient is preset to 1. It also takes a list of nodes for which gradients are to be computed and returns a list of gradient nodes corresponding to each node in the input list.

Returning to our earlier example, once you have implemented the gradients function, you can use it to calculate the gradients of $y$ with respect to $x_1$ and $x_2$. This is done by running:

```x1_grad, x2_grad = ad.gradients(output_node=y, node=[x1, x2])```

to obtain the respective gradients. Following this, you can set up an Evaluator with nodes y, x1_grad, and x2_grad. This allows you to use the Evaluator.run method to compute both the output value and the gradients for the input nodes.

Before you start working on the assignment, let's clarify how operations (ops) work. Within auto_diff.py, each op is equipped with three methods:

```__call__(self, **kwargs) -> Node```:

*  accepts input nodes (and attributes), creates a new node utilizing this op, and returns the newly created node.

```compute(self, node: Node, input_values: List[torch.Tensor])-> torch.Tensor```

*  processes the specified node along with its input values and delivers the resultant node value.

```gradient(self, node: Node, output_grad: Node) -> List[Node]```

*  receives a node and its gradient node, returning the partial adjoint nodes for each input node.

In essence, the `Op.compute` method is responsible for calculating the value of an individual node based on its inputs, while the `Evaluator.run` function computes the value of the entire graph's output based on the graph's inputs. The `Op.gradient` method is designed to construct the backward computational graph for an individual node, whereas the gradients function builds the backward graph for the entire graph. Accordingly, your implementation of `Evaluator.run `should effectively utilize the compute method from op, and your implementation of the gradients function should make use of the gradient method provided by op.

## **Question 1: Auto Diff Library (45 pt)**

### Part 1: Operators (25 pt)
In this problem, you will finish several operators in the autodiff.py class. Your goal is to implement the compute (forward) and gradient (backwards) function for these operators, with a few examples provided to you, such as `AddOp`, `AddByConstOp`, `MulOp` and `MulByConstOp`, and more. We have also implemented several other functions, such as `BroadcastOp` which will be useful in Question 3, but you can ignore them for now.

The list of operators that you will need to implement are:

*   `DivOp`
*   `DivByConstOp`
*   `TransposeOp`
*   `ReLUOp`
*   `SqrtOp`
*   `PowerOp`
*   `MeanOp`
*   `MatMulOp`
*   `SoftmaxOp`
*   `LayerNormOp`

You will find that most of your time will be spent on Matmul, SoftMax, and the LayerNorm operators. In turn, these 3 operators alone will be half of the points for this section.

### Part 2: Evaluator (20 pt)
You will also complete the entire `Evaluator` class. The details of the `Evaluator` class and what we want to achieve are given in the introduction section. We have provided a framework, including a topological sort method that you may choose to implement to help you more easily complete this class.

There are several tests provided to ensure your operators are working.

To run our sample testing library, you can use the commands:

```pytest tests/test_auto_diff_node_forward.py```

```pytest tests/test_auto_diff_node_backward.py```

```pytest tests/test_auto_diff_graph_forward.py```

```pytest tests/test_auto_diff_graph_backward.py```

Feel free to also edit these test files to include your own test cases!

**NOTE: These tests are not fully comprehensive, so we highly encourage you to write your own tests and ensure that your implemented operators are working.** You may find yourself going back and forth between this part and question 2 to ensure your operators are fully correct.


In [7]:
!pytest tests/test_auto_diff_node_forward.py
!pytest tests/test_auto_diff_node_backward.py
!pytest tests/test_auto_diff_graph_forward.py
!pytest tests/test_auto_diff_graph_backward.py

============================= test session starts ==============================
platform linux -- Python 3.10.16, pytest-8.3.4, pluggy-1.5.0
rootdir: /mnt/d/project/cse234-w25-PA/pa1
plugins: anyio-4.8.0
collected 13 items                                                             

tests/test_auto_diff_node_forward.py .............                       [100%]

============================== 13 passed in 0.87s ==============================
============================= test session starts ==============================
platform linux -- Python 3.10.16, pytest-8.3.4, pluggy-1.5.0
rootdir: /mnt/d/project/cse234-w25-PA/pa1
plugins: anyio-4.8.0
collected 12 items                                                             

tests/test_auto_diff_node_backward.py ............                       [100%]

============================== 12 passed in 0.82s ==============================
============================= test session starts ==============================
platform linux -- Pytho

## **Question 2: Transformer & Training (40 pt)**

In this assignment, you will implement core components of a Transformer model using an automatic differentiation system. You'll build essential neural network components including linear layers, attention mechanisms, and decoder layers.

Note that, for the sake of simplicity, the transformer that we are requiring you to implement does **NOT** include residual connections or multiple decoder heads.

### Provided Operations
Your implementation should only use operations THAT YOU IMPLEMENTED from the **auto_diff** module, not pre-built torch methods. Any transformers implemented explicitly with the torch autodiff library will be given zero credit.

### Background

The Transformer architecture, introduced in "[Attention Is All You Need](https://arxiv.org/abs/1706.03762)" (Vaswani et al., 2017)," revolutionized natural language processing. This assignment focuses on implementing key components using our custom automatic differentiation framework.

### Part 1: Linear Layer (5 pt)

Implement a linear transformation module that performs: $$output = input @ weight + bias$$

### Part 2: Single-Head Attention (10 pt)

Implement single-head attention mechanism that computes scaled dot-product attention:

$$Q = input @ W_Q, K = input @ W_K, V = input @ W_V$$

$$A = Softmax (\frac{Q@K^T}{\sqrt{d_k}})$$

### Part 3: Encoder Layer (10 pt)
Implement a encoder layer that combines self-attention and feed-forward networks.

### Part 4: Training (15 pt)

In transformer.py

1. Complete the ```transformer``` function. This function should include Part 1, 2 and 3 to create the full forward pass of a transformer layer.
2. Complete the ```softmax_loss``` function. This function takes in predicted logits and ground truth labels to compute a loss.
3. Complete the ```sgd_epoch``` function. This function completes one epoch of training and updates model parameters.

Then you will be able to run ``` python transformer.py ``` and train a single layer ViT on the MNIST dataset!

After training, you should observe that the test accuracy should be **at least 50%.** Note that this accuracy is not that high! It is not required to implement any residual connections or multi-head attention, so we don't expect it to perform extremely well.

#### BroadcastOp Deep Dive
In question 1, we implemented `BroadcastOp` for you, takes a tensor of a given shape and "expands" it so that it matches a larger, target shape, enabling element-wise operations between tensors of different but compatible dimensions.

A quick example: Suppose you have a tensor \( x \) of shape \([1, 5]\). You want to perform
an element-wise operation that requires a shape \([3, 5]\). You’d call:

```python
broadcast(x, input_shape=[1, 5], target_shape=[3, 5])
```

Forward Pass:
x is shape [1, 5] and is expanded to shape [3, 5]. )t creates a view that repeats the single row across the new dimension until the shape matches [3, 5].

Backward Pass:
During backpropagation, the gradient at shape [3,5] must be “collapsed” back to [1,5], so you sum along the newly expanded dimension(s). That way, if each of the 3 rows contributed to the gradient, those contributions are correctly aggregated into the single row of the original tensor shape.


In [1]:
!python transformer.py

Avg_loss: tensor([9.1286e-07, 8.0360e-07, 1.4553e-06, 8.4950e-07, 7.5689e-07, 6.5398e-07,
        7.6308e-07, 7.8390e-07, 1.2116e-06, 5.5913e-07, 8.3119e-07, 7.1768e-07,
        8.4950e-07, 5.6754e-07, 8.4950e-07, 7.2092e-07, 7.3867e-07, 7.1768e-07,
        7.6418e-07, 7.2612e-07, 9.6971e-07, 4.1947e-09, 6.2406e-07, 1.8725e-07,
        8.0371e-07, 6.9076e-07, 6.8451e-07, 3.3944e-07, 9.3123e-07, 8.1897e-07,
        3.8388e-09, 8.2530e-07, 7.5651e-07, 4.4369e-08, 7.5087e-07, 1.0784e-06,
        7.6298e-07, 4.6174e-07, 7.6418e-07, 2.6984e-07, 8.0360e-07, 7.3520e-07,
        6.5754e-07, 8.6957e-08, 5.3948e-07, 7.6394e-07, 6.8339e-07, 7.2004e-07,
        8.4950e-07, 4.0601e-07], dtype=torch.float64)
Epoch 0: test accuracy = 0.1966, loss = tensor([9.1286e-07, 8.0360e-07, 1.4553e-06, 8.4950e-07, 7.5689e-07, 6.5398e-07,
        7.6308e-07, 7.8390e-07, 1.2116e-06, 5.5913e-07, 8.3119e-07, 7.1768e-07,
        8.4950e-07, 5.6754e-07, 8.4950e-07, 7.2092e-07, 7.3867e-07, 7.1768e-07,
        7.6418e-

## **Question 3: Implementing Fused Operations (15pt + 10pt extra credit :D)**

In this final problem, you will implement fused operators for common deep-learning computations. This will act as an introduction to PA2, where we will aim to implement more fused operations, tiling, and other optimization techniques.

 Fused operators combine multiple operations into a single kernel to improve computational efficiency by reducing memory bandwidth usage and kernel launch overhead. You will implement two key fused operations:

1. Fused Matrix Multiplication + Layer Normalization
2. Fused Matrix Multiplication + Softmax

These operations are commonly used in transformer architectures and other deep-learning models. Your implementation will help you understand both the mathematical foundations and practical considerations of operator fusion.

### Task 1: Implement MatMulLayerNormOp (10 pt)

Complete the `compute()` and `gradient()` methods in the `MatMulLayerNormOp` class. The operator should:

* Perform matrix multiplication of inputs A and B
* Apply layer normalization along the specified dimensions  
* Compute correct gradients for backpropagation
* No need to implement `elementwise_affine` (check [torch.layer_norm](https://pytorch.org/docs/stable/generated/torch.nn.LayerNorm.html))

### Task 2: Implement MatMulSoftmaxOp (optional - 10 pts extra credit!)

Complete the `compute()` and `gradient()` methods in the `MatMulSoftmaxOp` class. The operator should:

* Perform matrix multiplication of inputs A and B
* Apply softmax along the specified dimension
* Compute correct gradients for backpropagation

### Task 3: Writeup (5 pt)
Create a text file with the name part3.txt that gives us a one to two paragraph explanation about:
* the intuition behind fused operators
* why it works for improving efficiency
* potential future improvements to these operators

### Testing

There are several tests provided to ensure your operators are working.

To run our sample testing library, you can use the commands:

```pytest tests/test_fused_ops.py```

To test the speed/performance compared to unfused operators

```python tests/test_fused_ops_perf.py```



In [9]:
!pytest tests/test_fused_ops.py
!python tests/test_fused_ops_perf.py

============================= test session starts ==============================
platform linux -- Python 3.10.16, pytest-8.3.4, pluggy-1.5.0
rootdir: /mnt/d/project/cse234-w25-PA/pa1
plugins: anyio-4.8.0
collected 4 items                                                              

tests/test_fused_ops.py ....                                             [100%]

============================== 4 passed in 0.83s ===============================

Running benchmark: 2D Matrix Multiplication

Testing shapes: (128, 64) @ (64, 128)

Testing MatMul + LayerNorm:

Forward Pass:
Fused:    0.118 ms ± 0.089 ms
Unfused:  0.140 ms ± 0.025 ms
Speedup:  1.19x

Backward Pass:
Fused:    0.252 ms ± 0.028 ms
Unfused:  0.313 ms ± 0.034 ms
Speedup:  1.24x

Testing MatMul + Softmax:

Forward Pass:
Fused:    0.124 ms ± 0.025 ms
Unfused:  0.122 ms ± 0.025 ms
Speedup:  0.98x

Backward Pass:
Fused:    0.190 ms ± 0.037 ms
Unfused:  0.193 ms ± 0.030 ms
Speedup:  1.02x

Testing shapes: (512, 256) @ (256, 512)

Test

## Submission

Congratulations, you have finished PA1!

We ask you to submit the entire CSE234-W25-PA1 folder as a zipped folder to Gradescope under the assignment 'PA1'. This includes all test files and any additional tests you may have written. Submission is due **Sunday, 2/9 at 11:59 PM.**

Please note that you have a total of **5 free late days** across all assignments. This means that you can choose to allocate these 5 free late days towards any assignments you choose, but you will not earn credit if your assignment is later than 5 days (without special request). So please turn your assignments in on time! 

As this is our first time running this specific PA1, if you have any feedback for the difficulty of this assignment, feel free to leave it in the `feedback.txt` file. Any responses will be much appreciated!